In [44]:
# Import dependencies
import praw
from psaw import PushshiftAPI
import pandas as pd
import datetime
import time
import matplotlib.pyplot as plt
import numpy as np
from config import client_id, client_secret, user_agent, db_password
from sqlalchemy import create_engine
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/adamweber/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [45]:
# Set up API credentials
reddit = praw.Reddit(client_id = client_id,
                     client_secret = client_secret,
                     user_agent = user_agent)
api = PushshiftAPI()

In [46]:
def scrape():
    
    submissions = list()
    query = api.search_comments(subreddit='cryptocurrency', q="bitcoin|btc", limit=1000)
    for element in query:
        submissions.append(element.d_)

    df = pd.DataFrame(submissions)
    df = df[['id', 'score', 'subreddit', 'body', 'created_utc']]
    df['datetime'] = df['created_utc'].map(lambda t: datetime.datetime.fromtimestamp(t))
    df = df.drop('created_utc', axis=1)
    df = df.sort_values(by='datetime')
    duplicateRowsDF = df[df.duplicated(['body'])]
    df = df.drop_duplicates(subset='body', keep="first")
    
    analyzer = SentimentIntensityAnalyzer()
    df['neg'] = df['body'].apply(lambda x:analyzer.polarity_scores(x)['neg'])
    df['neu'] = df['body'].apply(lambda x:analyzer.polarity_scores(x)['neu'])
    df['pos'] = df['body'].apply(lambda x:analyzer.polarity_scores(x)['pos'])
    df['compound'] = df['body'].apply(lambda x:analyzer.polarity_scores(x)['compound'])
    return df

In [47]:
df = scrape()

In [48]:
average = df.compound.mean()
average

0.19927373737373744